In [4]:
from math import *
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
from numpy import linalg as LA
from random import randint
import cmath
a=1.42
t=1
gruneisen=-2

def deltat1(uxx,uxy,uyy):
    return (gruneisen/4)*(uxx+3*uyy+2*np.sqrt(3)*uxy)

def deltat2(uxx,uxy,uyy):
    return (gruneisen/4)*(uxx+3*uyy-2*np.sqrt(3)*uxy)

def deltat3(uxx,uxy,uyy):
    return gruneisen*uxx

#points_per_side can be changed to any number
points_per_side = 100 #Running for 1000 pts takes 7 minutes 30 seconds. But it seems that the Berry Curv converges bw 56. -56 which is reasonable and similar to my analytic case.
t1=1+deltat1(0.0,0.0,0)
t2=1+deltat2(0.0,0.0,0)
t3=1+deltat3(0.0,0.0,0)

delta=0.2
kbt=25.7*(10**(-3)) #energy room temp


# points = []

# Loop to build hexagonal lattice
# for n in range(points_per_side):
#     x, y = n, 0
#     points.append((x, y))
#     for i in range(n):
#         x -= 0.5
#         y += sqrt(3) / 2
#         points.append((x, y))
#     for i in range(n):
#         x -= 1
#         points.append((x, y))
#     for i in range(n):
#         x -= 0.5
#         y -= sqrt(3) / 2
#         points.append((x, y))
#     for i in range(n):
#         x += 0.5
#         y -= sqrt(3) / 2
#         points.append((x, y))
#     for i in range(n):
#         x += 1
#         points.append((x, y))
#     for i in range(n):
#         x += 0.5
#         y += sqrt(3) / 2
#         points.append((x, y))

# points=np.array(points)
# kx=points[:,0]
# ky=points[:,1]

mu = 1

x = np.arange(-5,5,0.01)
y = np.arange(-5,5,0.01)

kx,ky=np.meshgrid(x,y)


#uxx002 = np.array([])





kd1=(a/2)*(((np.sqrt(3))*ky)+kx)
kd2=(a/2)*(((-np.sqrt(3))*ky)+kx)
kd3=-a*kx

hx=(-t1*np.cos(kd1))-(t2*np.cos(kd2))-(t3*np.cos(kd3))
hy = (t1*np.sin(kd1))+(t2*np.sin(kd2))+(t3*np.sin(kd3))

hxx=(a/2)*(t1*np.sin(kd1)+t2*np.sin(kd2)+2*t3*np.sin(-kd3))
hyx=(a/2)*(t1*np.cos(kd1)+t2*np.cos(kd2)-2*t3*np.cos(-kd3))

hyy=np.sqrt(3)*(a/2)*(t1*np.cos(kd1)-t2*np.cos(kd2))
hxy=np.sqrt(3)*(a/2)*(t1*np.sin(kd1)-t2*np.sin(kd2))


def main(hx, hy):

    # First, generate the matrices of the form
    # [[kx[i], ky[i]], [kx[i], ky[i]]]
    # NOTE: It is assumed that kx and ky are numerical lists of the
    #     same length
    matrices = [np.array([[delta,hx[i]-complex(0,1)*hy[i]], [hx[i]+complex(0,1)*hy[i],-delta]],dtype=float)
                for i in range(len(kx))]
    
    #print(matrices[0])
    # initialize lists to catch eigenvalue and eigenvector pairs
    eval_pairs = []
    evec_pairs = []

    for M in matrices: # for each matrix...
        # Get its eigenvalues and eigenvectors
        (temp_eval_pair, temp_evec_pair) = LA.eig(M)
        # Store the eigenvalues and eigenvectors separately
        eval_pairs.append(temp_eval_pair)
        evec_pairs.append(temp_evec_pair)
        
    # eval_list_1 is the list of first eigenvalues
    # eval_list_2 it eh list of second eigenvalues
    eval_list_1 = [l[0] for l in eval_pairs]
    eval_list_2 = [l[1] for l in eval_pairs]
    
    #eval_list_2 is positive eigenvalues
    
    
    #print(eval_list_2[0])
    #print(eval_list_2[0:5])

    # evec_list_1 is the list of first eigenvectors
    # evec_list_2 is the list of second eigenvectors
    # Note that the vectors are already normalized by eig function
    # Note also the way these are generated --- the ones in evec_list_1
    #    come from the first COLUMN of the entry of evec_pairs, according
    #    to the spec of the numpy.eig function
    evec_list_1 = [np.matrix([[u[0][0]], [u[1][0]]]) for u in evec_pairs]
    evec_list_2 = [np.matrix([[u[0][1]], [u[1][1]]]) for u in evec_pairs]
    
    #print(evec_list_1[0])
    #print(evec_list_2[0])
    # Generate a 2x2 matrix A, with integer entries chosen randomly
    # between -99 and 99
    vx = [np.array([[0,hxx[i]-complex(0,1)*hyx[i]],[hxx[i]+complex(0,1)*hyx[i],0]],dtype=float) for i in range(len(kx))] 
    #print(vx[1])
    
    vy = [np.array([[0,hxy[i]-complex(0,1)*hyy[i]],[hxy[i]+complex(0,1)*hyy[i],0]],dtype=float) for i in range(len(kx))]
    
    #A = np.random.randint(-99, 100, (2, 2))

    # Compute the list of products
    # We first compute the tranpose of the entry in evec_list_2, to get
    #     a row-vector form of u_2
    # Then we multiply by A, and then multiply by u_1
    # This gives (1x2) @ (2x2) @ (2x1), which results in a 1x1 result
    #     (i.e., a number)
    # Those 1x1 results are stored in prod_list; they are flattened into
    #     numbers in prod_list_simple
    # The tuple(matrices, eval_list_1, eval_list_2,
    #           evec_list_1, evec_list_2, A, prod_list_simple)
    #     is returned by this function, for later use if necessary
    prod_list_12 = [evec_list_1[k].getH() @ vx[k] @ evec_list_2[k]
                  for k in range(len(evec_list_1))]
    
    #Changed from _1 * _2 to _2*_1 to make it same order as thesis with cond band etc. since eval_2 is the cond band energy.
    
    #print(evec_list_2[1])
    #print(evec_list_1[1].getH())
    #print(vx[1])
    #print(np.shape(prod_list_12[0]))
    #print(prod_list_12[1])
    #print(np.shape(prod_list_12[0]))
    #print(prod_list_12[0][1])
    #print(prod_list_12)
    #print()
    prod_list_21 = [evec_list_2[k].getH() @ vy[k] @ evec_list_1[k]
                  for k in range(len(evec_list_1))]

#Changed from _2*_1 to 1_* _2 for same reason, now consistent with eigenvalues.




    #prod_list_12_simple = [P[0][0] for P in prod_list_12]
    #prod_list_21_simple = [P[0][0] for P in prod_list_21]
    #print(prod_list_12[0:5])
    
    Mx12=(np.asarray(prod_list_12).flatten())
    
    My21 =(np.asarray(prod_list_21).flatten())
    
    
    #print(prod_list_12_simple)

    return((matrices, eval_list_1, eval_list_2, evec_list_1,
            evec_list_2, Mx12, My21))

(m, eval_1, eval_2, evec_1, evec_2,u1u2prods,u2u1prods)=main(hx,hy)



matrixprod = u1u2prods*u2u1prods

#print(matrixprod)

eig1 = (eval_1)
eig1array=np.asarray(eig1)

eig2 = (eval_2)
eig2array=np.asarray(eig2)


Berryc = (-2*np.imag(matrixprod)/((np.real(eig2array)-np.real(eig1array))**2))

# eigenvals=np.sqrt((hx**2)+(hy**2)+(delta**2)) #this is analytic tho

denominatorf0 = (((np.exp((np.real(eig1array)-mu)/kbt)+1)**2)*np.real(eig1array)*kbt)

df0hx = (-2*hx*np.exp((np.real(eig1array)-mu)/(kbt)))/denominatorf0

df0hy = (-2*hy*np.exp((np.real(eig1array)-mu)/(kbt)))/denominatorf0

integranddipolewrtx=((df0hx*hxx + df0hy*hyx)*Berryc)/(4*((np.pi)**2))

integranddipolewrty=((df0hy*hyy + df0hx*hxy)*Berryc)/(4*((np.pi)**2))

dipolex = -(sum(integranddipolewrtx))/(len(integranddipolewrtx))

dipoley = -(sum(integranddipolewrty))/(len(integranddipolewrty))



plt.pcolormesh(kx,ky,Berryc)

print(dipolex)
print(dipoley)

#Uniaxial 0 to 0.1 below:
    
# strain=[0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]
#Dy = [0,3.532530012370234e-05,6.623078032036314e-05,0.00010062156896876305,0.00014596778548100627,0.0001963698238264582,0.00023848121366110473,0.00026904838172267216,0.00029988350607558875,0.0003427819336875057,0.0003945308277964405]

#Dy=[0,-3.462525230075699e-05,-6.83181173234318e-05,-0.0001006731938970472,-0.00013262180517476551,-0.000164287077103341,-0.0001949928490227373,-0.00022440440380766454,-0.00025297243060914113,-0.00028097597643073405,-0.0003080418587294074]

# Dx = [0,0,0,0,0,0,0,0,0,0,0]
# plt.plot(strain,Dy,label='$D_{yz}$')
# plt.plot(strain,Dy,'o')
# plt.plot(strain,Dx,label='$D_{xz}$')
# plt.xlabel('$u_{xx}$',fontsize=14)
# plt.ylabel('Berry curvature dipole $(Å)$',fontsize=14)
# plt.title('Plot of $D_{yz}$ and  $D_{xz}$ against uniaxial strain $u_{xx}$ ', fontsize=14)
# plt.legend(fontsize=14, loc='center right')

#Shear below

# shear= [0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]
# Dx=[0,7.437743952030791e-05,0.0001460158550116842,0.00021440541186551454,0.00028215715597378033,0.00035325144879223835,0.00042921919659611433,0.0005070125721040162,0.0005834049134339892,0.0006623534579674819,0.0007509773282801984]
# Dy=[0,2.317444892280869e-06,9.048789609525814e-06,1.9270751443907536e-05,3.105390852729272e-05,4.2313110170996315e-05,5.336113587888594e-05,6.870473220321715e-05,9.353651049191126e-05,0.00012630097058837942,0.00016079837649393778]
# plt.plot(shear,Dx,label='$D_{xz}$')
# plt.plot(shear,Dy,label='$D_{yz}$')
# plt.plot(shear,Dx,'o')
# plt.plot(shear,Dy,'o')
# plt.xlabel('$u_{xy}$=$u_{yx}$',fontsize=14)
# plt.ylabel('Berry curvature dipole $(Å)$',fontsize=14)
# plt.title('Plot of $D_{yz}$ and  $D_{xz}$ against shear strain $u_{xy}$=$u_{yx}$  ', fontsize=14)
# plt.legend(loc='center right', fontsize=14)

#Chemical potential for uxx=0.02 below (mu between 0 and 3.4)

#uxx000=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
#uxx002=-1*np.array([0,1.951104365909785e-06,0.0001257379874247214,0.00020173932794544477,0.00042972259481817383,0.0004583943004475531,0.0004516672606993754,0.00041779027723637716,0.000397869727141432,0.0004086925374836021,0.00055587000408615,0.0003193968323026458,0.00019641303908078813,0.00012761557130237181,8.952928023000422e-05,6.623078032036314e-05,5.068336080806081e-05,3.955874399486233e-05,3.1231158225666415e-05,2.484946284461217e-05,1.9904710798467124e-05,1.6035980966247717e-05,1.2888082710291797e-05,1.0183505284794122e-05,7.811303619148229e-06,5.910919378327326e-06,4.422694591784944e-06,3.1466253957373544e-06,2.0956038290945053e-06,1.2043382527612955e-06,4.597981322584721e-07,0,0,0,0])
#muu =np.arange(0,3.5,0.1)


#Chemical potential for uxx=0.04 below (mu between 0 and 3.4)

#uxx004=-1*np.array([0,7.365718205313702e-06,0.00023528459254734883,0.0007372084944180105,0.0007696018205738692,0.0007949642588323372,0.000727941377700548,0.000688563395782744,0.0006720098386235032,0.0007193392691153569,0.0009456952919511467,0.0006045505359669873,0.00037189864053610675,0.00025641926497648615,0.00019016324553473965,0.00014596778548100627,0.00011349929619008672,8.872478558059287e-05,6.961958751438863e-05,5.479332241414763e-05,4.31736049551296e-05,3.3985774256756494e-05,2.6634309327746942e-05,2.0888061218730023e-05,1.6491967730814532e-05,1.2946961974945778e-05,9.80604287431614e-06,7.174560761155607e-06,5.021920826806473e-06,3.202266652458772e-06,1.6539554294130645e-06,0,0,0,0])

#Chemical potential for uxx=0.06 below

#uxx006=-1*np.array([0,1.5066991451246535e-05,0.00030959657668018893,0.001290267114419585,0.0014430178403009827,0.001337680071740637,0.001222902647518078,0.0011301091747829,0.0011098469317720016,0.0011860778381154854,0.0014310273096098153,0.001131522789676318,0.0006656647869000778,0.00045477494542159476,0.0003245521458313406,0.00023848121366110473,0.00017849284595698682,0.00013599985638731388,0.00010545623899196886,8.299186982313305e-05,6.604436775612462e-05,5.296381138230331e-05,4.2609030251753385e-05,3.413036624892985e-05,2.6958501878791125e-05,2.085900556215785e-05,1.5964683621559133e-05,1.2081787460411927e-05,8.73609015795323e-06,5.945297186415974e-06,3.5686071766678717e-06,0,0,0,0])

#Chemical potemtial for uxx=0.08 below


#uxx008 = -1*np.array([0,1.7553015120744635e-05,0.0004181297420780277,0.0015728485144568289,0.0019676791924280726,0.0018112275984035395,0.001658789090572123,0.0015443530111485704,0.0014794558318975042,0.0015589632757092957,0.0017411874537344419,0.0014485415228825732,0.0008509881847682126,0.0005862965724556911,0.00041065571065780404,0.00029988350607558875,0.00022664131104757848,0.0001760848109855086,0.00013939339485854243,0.00011148775131040874,8.955827657524029e-05,7.19925727958518e-05,5.782335143812924e-05,4.6243381898942745e-05,3.6917350102956993e-05,2.9453929723262767e-05,2.3266723018345204e-05,1.7822607842207008e-05,1.319004932409904e-05,9.394119052220127e-06,6.158132161155012e-06,0,0,0,0])

#Chemical potential for uxx=0.1 below

#uxx01= -1*np.array([0,1.7292564709164897e-05,0.0005752450753162106,0.002023282051787011,0.002273652101721545,0.0021298770958704163,0.0019227580545022266,0.0017974952562110568,0.0017560292481328395,0.0018895774391101022,0.0020301617039101003,0.0017312457606793862,0.0011380935601941898,0.0007503436147827895,0.0005330429144880021,0.0003945308277964405,0.0003018352214585827,0.00023604702957760789,0.00018689015255744033,0.00014898970023995254,0.00011932354709771493,9.591591763139644e-05,7.732703258361513e-05,6.233847899302488e-05,5.005798895670147e-05,3.9800806485535735e-05,3.116846230873133e-05,2.410937254229871e-05,1.8411215902318295e-05,1.3525136855705327e-05,0,0,0,0,0])

# plt.plot(muu,uxx000,'--',label='$u_{xx}=0$')
# plt.plot(muu,uxx002,label='$u_{xx}=0.02$')
# plt.plot(muu,uxx004,label='$u_{xx}=0.04$')
# plt.plot(muu,uxx006,label='$u_{xx}=0.06$')
# plt.plot(muu,uxx008,label='$u_{xx}=0.08$')
# plt.plot(muu,uxx01,label='$u_{xx}=0.1$')
# plt.xlabel('Chemical potential $\mu$ (eV)',fontsize=14)
# plt.ylabel('$D_{yz}$ $(Å)$',fontsize=14)
# plt.legend(fontsize=14,loc='upper right',bbox_to_anchor=(1.4, 0.9))
# plt.title('Plot of $D_{yz}$ against Chemical potential $\mu$ for different uniaxial strains $u_{xx}$  ', fontsize=14)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 2) + inhomogeneous part.